### Import necessary packages

In [62]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC






### Load the Twitter_stock_delay2.csv dataset

In [23]:
twitter_stock_2=pd.read_csv("../Data/Twitter_stock_delay2.csv")


In [24]:
df=pd.DataFrame([twitter_stock_2["Tweets_clean_more"],twitter_stock_2["Label_delayed_2Days"]]).T
df


,Tweets_clean_more,Label_delayed_2Days
0,to super clear i dont wish could i mandat anyt...,Decrease
1,ye get regulatori pushback may avail region go...,Increase
2,dome barrel weld made bar pretti good bar need...,Decrease
3,the exec conf room tesla use call denali i dec...,Decrease
4,sad hear david bowi die he amaz,Increase
...,...,...
1415,type peopl world understand binari amp will s...,Decrease
1416,excel explan diff betw get orbit v space sugge...,Decrease
1417,a la guerr comm à la guerr haha and start twee...,Decrease
1418,meet build call just crazi japanes game show ...,Increase


### Get our train and val dataset

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df["Tweets_clean_more"], df["Label_delayed_2Days"], test_size=0.2,random_state=0)

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train shape:  (1136,)
X_val shape:  (284,)
y_train shape:  (1136,)
y_val shape:  (284,)


In [85]:
y_train

863      Increase
654      Increase
299     No Change
539      Increase
981      Increase
          ...    
763      Decrease
835      Decrease
1216     Decrease
559      Increase
684      Decrease
Name: Label_delayed_2Days, Length: 1136, dtype: object

### Check the structure of our dataset

In [86]:
print('Train Class Proportion:\n', y_train.value_counts() / len(y_train) * 100)
print('\nValidation Class Proportion:\n', y_val.value_counts() / len(y_val) * 100)

Train Class Proportion:
 Decrease     48.679577
Increase     48.151408
No Change     3.169014
Name: Label_delayed_2Days, dtype: float64

Validation Class Proportion:
 Increase     49.295775
Decrease     48.943662
No Change     1.760563
Name: Label_delayed_2Days, dtype: float64


### Countvectorize our dataset

In [89]:

# COUNTVECTORIZER
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
x_train = vect.fit_transform(X_train)
x_test = vect.transform(X_val)

### Get our basic model and voting ensemble model

In [94]:

from sklearn.neighbors import KNeighborsClassifier
KNN_clf = KNeighborsClassifier(n_neighbors=7,p=2)
rf_clf = RandomForestClassifier(n_estimators = 100,random_state=000)
svm_clf =  SVC(C=10,kernel='rbf',probability=True)

voting_clf = VotingClassifier(estimators=[ ('svc', svm_clf),
                                           ('rf', rf_clf), 
                                           ('svc1', KNN_clf)],voting='hard')

for clf in ( svm_clf,rf_clf, KNN_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print( clf,accuracy_score(y_val, y_pred))

SVC(C=10, probability=True) 0.5774647887323944
RandomForestClassifier(random_state=0) 0.5070422535211268
KNeighborsClassifier(n_neighbors=7) 0.5211267605633803
VotingClassifier(estimators=[('svc', SVC(C=10, probability=True)),
                             ('rf', RandomForestClassifier(random_state=0)),
                             ('svc1', KNeighborsClassifier(n_neighbors=7))]) 0.5633802816901409


Accuracy of the hard voting ensemble model shown above.